In [12]:
import requests

url = "http://crmgpu6-10042:37991/image/api/resume/parse"

payload = {}
files = [
    ('resume', ('HR2.pdf', open("C:/Users/shari/Desktop/Zoho/HR/HR2.pdf", 'rb'), 'application/pdf'))
]
headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

In [2]:
data = response.json()

In [3]:
parsed_data = data['parsed_json']

In [4]:
from sentence_transformers import SentenceTransformer, util

# Load the model
model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

sections = []

for exp in parsed_data.get('work_experiences', []):
    work_desc = exp.get('work_description')
    if isinstance(work_desc, list):
        work_desc = " ".join(work_desc)
    if work_desc and work_desc.strip():
        sections.append(work_desc)
for desc in [parsed_data['primary_information'].get('profile_description', '')]:
    if desc and desc.strip():
        sections.append(desc)
for ach_desc in parsed_data.get("achievements", []):
    if ach_desc:  
        sections.append(ach_desc)
for section in ["certifications", "projects"]:
    if parsed_data.get(section):
        sections.extend(parsed_data[section])  
    else:
        sections.append(f"No {section} available in the resume")
Skills = []

for skill in parsed_data.get('skills', {}).get('hard_skills', []):
    Skills.append(skill.get('parsed_value'))

for skill in parsed_data.get('skills', {}).get('soft_skills', []):
    Skills.append(skill.get('parsed_value'))

In [5]:
work_embeddings = model.encode(sections, convert_to_tensor=True)   
skill_embeddings = model.encode(Skills, convert_to_tensor=True)  


similarity = util.cos_sim(work_embeddings, skill_embeddings)
print(sections)

[" Recruitment, hiring and employee relations in departments\n Sourcing CVs from various channels such as referencing, company's own database, Job Portals\n i.e. Monster, Naukri, and networking tools such as LinkedIn.\n Planning and structuring of job advertisements on job portals.\n Scheduling interview, fallow up to get feedback, negotiating salary.\n Completion and coordination for employment of application formality.\n Collecting necessary documents for hiring process.\n Assisted payroll administrator with completing bi- monthly payroll.\n Input new employee Information Into the system.\n Prepared detail reports for management.\n\n Prepare MIS joining new employees left in excel sheet\n Prepare MIS employee (Attendance, leave &OT records)", ' To associate myself with the organization, as an executive and be a target- oriented member,\n achieve organization mission and objective and help the organization in decision- making', 'No certifications available in the resume', {'title': 'E

In [6]:
import pandas as pd
work_labels = [f'w_{i+1}' for i in range(len(sections))]
print(work_labels)

['w_1', 'w_2', 'w_3', 'w_4']


In [7]:
df = pd.DataFrame(similarity, index=work_labels, columns=Skills)
df

MS Office     Excel      Word   Outlook  Power Point  Windows 7  \
w_1   0.369505  0.343590  0.269928  0.326498     0.308439   0.226020   
w_2   0.421794  0.390655  0.413454  0.455721     0.377497   0.364541   
w_3   0.465666  0.425422  0.472463  0.408873     0.462695   0.442670   
w_4   0.358537  0.356990  0.347298  0.326997     0.341154   0.314366   

     windows 8  windows9 and latest version  Effective Communication Skills  \
w_1   0.227848                     0.216978                        0.340173   
w_2   0.351680                     0.325779                        0.435912   
w_3   0.440028                     0.456610                        0.463957   
w_4   0.320602                     0.281020                        0.376648   

     Excellent Interpersonal Skills  ...  Motivation  Professionalism  \
w_1                        0.341544  ...    0.322577         0.308143   
w_2                        0.468770  ...    0.556330         0.475692   
w_3                        0.501736  ...    0.442330         0.511292   
w_4                        0.397253  ...    0.412667         0.392211   

     Hardworking  committed to excellence  Quick learner  \
w_1     0.339058                 0.271045       0.259775   
w_2     0.432522                 0.563285       0.365842   
w_3     0.452072                 0.501550       0.480942   
w_4     0.422437                 0.403050       0.335919   

     Creating a positive work environment  Self- motivated   honesty  \
w_1                              0.411403         0.248118  0.261952   
w_2                              0.522139         0.548276  0.426608   
w_3                              0.466234         0.472973  0.420918   
w_4                              0.499148         0.388306  0.357252   

      loyalty  punctuality  
w_1  0.261312     0.271960  
w_2  0.521892     0.441359  
w_3  0.449279     0.424598  
w_4  0.408977     0.372178  

[4 rows x 21 columns]

In [10]:
df = pd.DataFrame(similarity, index=work_labels, columns=Skills)
average = df.mean()

In [11]:
sorted_skills = average.sort_values(ascending=False)
sorted_skills

Creating a positive work environment    0.474731
committed to excellence                 0.434733
Motivation                              0.433476
Excellent Interpersonal Skills          0.427326
Professionalism                         0.421835
Self- motivated                         0.414418
Hardworking                             0.411522
loyalty                                 0.410365
Effective Communication Skills          0.404173
MS Office                               0.403875
Outlook                                 0.379522
Excel                                   0.379164
punctuality                             0.377524
Word                                    0.375786
Power Point                             0.372446
Ability to deal with pressure           0.371203
honesty                                 0.366682
Quick learner                           0.360619
Windows 7                               0.336899
windows 8                               0.335040
windows9 and latest 